# Abstractive

<div class="alert alert-info">

This tutorial is available as an IPython notebook at [Malaya/example/augmentation-abstractive](https://github.com/huseinzol05/Malaya/tree/master/example/augmentation-abstractive).
    
</div>

<div class="alert alert-warning">

Results generated using stochastic methods.
    
</div>

In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = ''
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

In [2]:
%%time

import malaya

/home/husein/.local/lib/python3.8/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


/home/husein/.local/lib/python3.8/site-packages/bitsandbytes/libbitsandbytes_cpu.so: undefined symbol: cadam32bit_grad_fp32
CPU times: user 2.9 s, sys: 3.74 s, total: 6.64 s
Wall time: 2.24 s


/home/husein/dev/malaya/malaya/tokenizer.py:214: FutureWarning: Possible nested set at position 3397
  self.tok = re.compile(r'({})'.format('|'.join(pipeline)))
/home/husein/dev/malaya/malaya/tokenizer.py:214: FutureWarning: Possible nested set at position 3927
  self.tok = re.compile(r'({})'.format('|'.join(pipeline)))


### Why augmentation

Let say you have a very limited labelled corpus, and you want to add more, but labelling is very costly.

So, text augmentation! We provided few augmentation interfaces in Malaya.

### List available HuggingFace models

In [3]:
malaya.augmentation.abstractive.available_huggingface

{'mesolitica/translation-t5-tiny-standard-bahasa-cased': {'Size (MB)': 139,
  'Suggested length': 1536,
  'ms-pasar ms chrF2++': 49.01,
  'en-pasar ms chrF2++': 45.29,
  'ms-manglish chrF2++': 37.55,
  'en-manglish chrF2++': 44.32,
  'from lang': ['en', 'ms', 'ind', 'jav', 'bjn'],
  'to lang': ['manglish', 'pasar ms']},
 'mesolitica/translation-t5-small-standard-bahasa-cased': {'Size (MB)': 242,
  'Suggested length': 1536,
  'ms-pasar ms chrF2++': 54.3,
  'en-pasar ms chrF2++': 51.88,
  'ms-manglish chrF2++': 39.98,
  'en-manglish chrF2++': 44.58,
  'from lang': ['en', 'ms', 'ind', 'jav', 'bjn'],
  'to lang': ['manglish', 'pasar ms']},
 'mesolitica/translation-t5-base-standard-bahasa-cased': {'Size (MB)': 892,
  'Suggested length': 1536,
  'ms-pasar ms chrF2++': 50.25,
  'en-pasar ms chrF2++': 49.26,
  'ms-manglish chrF2++': 38.41,
  'en-manglish chrF2++': 43.38,
  'from lang': ['en', 'ms', 'ind', 'jav', 'bjn'],
  'to lang': ['manglish', 'pasar ms']},
 'mesolitica/translation-nanot5-ti

### Load HuggingFace model

```python
def huggingface(
    model: str = 'mesolitica/translation-t5-small-standard-bahasa-cased',
    force_check: bool = True,
    **kwargs,
):
    """
    Load HuggingFace model to abstractive text augmentation.

    Parameters
    ----------
    model: str, optional (default='mesolitica/translation-t5-small-standard-bahasa-cased')
        Check available models at `malaya.augmentation.abstractive.available_huggingface`.
    force_check: bool, optional (default=True)
        Force check model one of malaya model.
        Set to False if you have your own huggingface model.

    Returns
    -------
    result: malaya.torch_model.huggingface.Translation
    """
```

In [7]:
model = malaya.augmentation.abstractive.huggingface(model = 'mesolitica/translation-nanot5-small-malaysian-cased')

### Generate

```python
def generate(self, strings: List[str], to_lang: str = 'ms', **kwargs):
    """
    Generate texts from the input.

    Parameters
    ----------
    strings : List[str]
    to_lang: str, optional (default='ms')
        target language to translate.
    **kwargs: vector arguments pass to huggingface `generate` method.
        Read more at https://huggingface.co/docs/transformers/main_classes/text_generation

        If you are using `use_ctranslate2`, vector arguments pass to ctranslate2 `translate_batch` method.
        Read more at https://opennmt.net/CTranslate2/python/ctranslate2.Translator.html?highlight=translate_batch#ctranslate2.Translator.translate_batch

    Returns
    -------
    result: List[str]
    """
```

In [5]:
string = 'saya suka makan ayam dan ikan'
string2 = 'saya sebenarnya tak suka sangat dekat lelaki tu, ketiak masam sebab tak mandi'
string3 = 'Perdana Menteri berkata, beliau perlu memperoleh maklumat terperinci berhubung isu berkenaan sebelum kerajaan dapat mengambil sebarang tindakan lanjut. Bagaimanapun, beliau yakin masalah itu dapat diselesaikan dan pentadbiran kerajaan boleh berfungsi dengan baik.'

In [8]:
model.generate([string, string2, string3], to_lang = 'pasar ms', max_length = 256)

spaces_between_special_tokens is deprecated and will be removed in transformers v5. It was adding spaces between `added_tokens`, not special tokens, and does not exist in our fast implementation. Future tokenizers will handle the decoding process on a per-model rule.


['aku suka makan ayam dan ikan',
 'aku sebenarnya tak suka sangat dekat laki tu, ketiak masam sebab tak mandi',
 'Perdana Menteri berkata, beliau perlu mendapatkan maklumat terperinci berhubung isu berkenaan sebelum kerajaan dapat mengambil sebarang tindakan lanjut.  Bagaimanapun, beliau yakin masalah itu dapat diselesaikan dan pentadbiran kerajaan dapat berfungsi dengan baik.']

### Good thing about HuggingFace

In `generate` method, you can do greedy, beam, sampling, nucleus decoder and so much more, read it at https://huggingface.co/blog/how-to-generate

In [9]:
outputs = model.generate(['rakyat memang tak suka awak pun'], to_lang = 'pasar ms',
                         max_length = 100, do_sample=True, top_k=100, top_p=0.95, temperature=0.7,
                         num_return_sequences=5)
outputs

['rakyat mmg tak suke u pun',
 'rakyat mmg tak suke awak pun',
 'rakyat memang tak suka awak pun',
 'rakyat mmg tak suka awak pun',
 'rakyat memang tak suka awak pun']

In [10]:
outputs = model.generate([string, string2, string3], to_lang = 'pasar ms',
                         max_length = 100, do_sample=True, top_k=100, top_p=0.95, temperature=0.7,
                         num_return_sequences=3)
outputs

['aku suka makan ayam ikan',
 'i suka makan ayam ikan',
 'suka makan ayam dan ikan',
 'aku memang tak suka sangat dekat lelaki tu  ketiak masam sebab tak mandi',
 'i paling x suka dekat lelaki tu  ketiak masam sbb x mandi',
 'aku sebenarnya tak suka sangat dekat laki tu, ketiak masam sebab tak mandi',
 '"Perdana Menteri berkata, beliau perlu mendapatkan maklumat terperinci berhubung isu berkenaan sebelum kerajaan dapat mengambil sebarang tindakan lanjut.  Bagaimanapun, beliau yakin masalah itu dapat diselesaikan dan pentadbiran kerajaan dapat berfungsi dengan baik."  "Terima kasih, Teks itu sudah cukup untuk ditafsirkan," tambah Perdana Menteri.',
 'Perdana Menteri berkata, beliau perlu mendapatkan maklumat terperinci berhubung isu berkenaan sebelum kerajaan dapat mengambil sebarang tindakan lanjut.  Bagaimanapun, beliau yakin perkara itu dapat diselesaikan dan pentadbiran kerajaan dapat berfungsi dengan baik.  http://www. great.com/arena..  [ Last edited by  yanhong_naz at 31-12-2007 

You can choose different generating method and still maintain the same polarity.

In [12]:
strings = [
    'bodoh betul kerajaan ni, mana kebebasan bersuara',
    'Perbincangan khas itu juga bertujuan bagi Seri Paduka mendapat pandangan Raja-Raja Melayu untuk membolehkan baginda membuat keputusan yang terbaik demi kepentingan dan kesejahteraan negara serta rakyat',
    'semalam saya buka tv tengok berita, katanya ada tanah runtuh',
    'i hate him so much, perangai teruk!'
]

In [13]:
outputs = model.generate(strings, to_lang = 'pasar ms',
                         max_length = 100, do_sample=True, top_k=100, top_p=0.95, temperature=0.8,
                         num_return_sequences=3)
outputs

['bodoh la pakatan harapan ni  mana kebebasan bersuara',
 'bodoh betul kerajaan ni,mana kebebasan bersuara',
 'bodoh betul kerajaan ni mana kebebasan bersuara',
 'Kongres khas itu turut bertujuan untuk Seri Paduka mendapat pandangan Raja-Raja Melayu bagi membolehkan baginda membuat keputusan yang terbaik demi kepentingan dan kesejahteraan negara dan rakyat',
 'Perbincangan khas itu juga bertujuan bagi Seri Paduka mendapat pandangan Raja-Raja Melayu bagi membolehkan baginda membuat keputusan yang terbaik demi kepentingan dan kesejahteraan negara serta rakyat',
 'Perbincangan khas itu juga bertujuan bagi Seri Paduka mendapat pandangan Raja-Raja Melayu bagi membolehkan baginda membuat keputusan yang terbaik demi kepentingan dan kesejahteraan negara serta rakyat https://t.co/M9BjB7UXpQ',
 'semalam aku bukak tv tgk berita katanya tanah runtuh',
 ' semalam bukak tv tengok berita dia kata tanah runtuh katanya',
 ' semalam aku buka tv tengok berita katanya tanah runtuh',
 'i hate him so much  

In [14]:
outputs = model.generate(strings, to_lang = 'pasar ms',
                         max_length = 100, do_sample=True, penalty_alpha=0.9, top_k=4, temperature=0.9,
                         num_return_sequences=3)
outputs

['bodoh betul kerajaan ni mana kebebasan bersuara',
 'bodoh la kerajaan ni, mana bebas bersuara',
 'bodoh betul gomen ni  mana freedom of speech',
 'Perbincangan khas itu juga bertujuan agar Seri Paduka mendapat pandangan Raja-Raja Melayu bagi membolehkan baginda membuat keputusan yang terbaik demi kepentingan dan kesejahteraan negara serta rakyat https://t.co/z6R9XVdZdD',
 'Perbincangan khas itu juga bertujuan untuk Seri Paduka mendapat pandangan Raja-Raja Melayu untuk membolehkan baginda membuat keputusan yang terbaik demi kepentingan dan kesejahteraan negara serta rakyat',
 'Perbincangan khas itu juga bertujuan bagi Seri Paduka mendapat pandangan Raja-Raja Melayu untuk membolehkan baginda membuat keputusan yang terbaik demi kepentingan dan kesejahteraan negara serta rakyat',
 'semalam aku buka tv tengok berita, kata ada tanah runtuh katanya',
 ' semalam saya buka tb tengok berita, katanya ada tanah runtuh',
 'semalam aku bukak tv tengok berita, ada tanah runtuh katanya ',
 'i hate h